In [ ]:
import csv
import sys
import pandas as pd
import ast

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
fileNameB = "subtask_b_train.tsv"
fileDirectoryB = "AdMIRe Subtask B Train/train"

dataB = pd.read_csv(fileDirectoryB + "/" + fileNameB, sep='\t')

In [ ]:
print(len(dataB))
print(len(dataB[dataB["sentence_type"]=="idiomatic"]))
print(len(dataB[dataB["sentence_type"]=="literal"]))

In [ ]:
dataB

In [ ]:
def sim_scores(current, sentences):
    # input = current line(example) & embeddings for sequence captions + image captions
    
    scores = {}
    # keys = image names
    # values = scores

    embeddings = model.encode(sentences)
    # compare embeddings for sequence1 & sequence 2 captions with all 4 images captions
    similarities = model.similarity(embeddings[:2], embeddings[2:])

    # average of similarity to sequence pictures
    score1 = (similarities[0][0].item() + similarities[1][0].item()) / 2
    scores[current["image1_name"]] = score1

    score2 = (similarities[0][1].item() + similarities[1][1].item()) / 2
    scores[current["image2_name"]] = score2

    score3 = (similarities[0][2].item() + similarities[1][2].item()) / 2
    scores[current["image3_name"]] = score3

    score4 = (similarities[0][3].item() + similarities[1][3].item()) / 2
    scores[current["image4_name"]] = score4
    
    return scores

In [ ]:
# evaluation - completion accuracy
# excluding prediction of idiomatic vs literal

data = dataB
#data = dataB[dataB["sentence_type"]=="literal"]
#data = dataB[dataB["sentence_type"]=="idiomatic"]

total_acc = 0

for i in range(len(data)):

    current = data.iloc[i]
    sentences = [current["sequence_caption1"],
                 current["sequence_caption2"], 
                 current["image1_caption"],
                 current["image2_caption"],
                 current["image3_caption"],
                 current["image4_caption"]]

    scores = sim_scores(current, sentences)

    # choose the image that is most similar to the sequence images
    pred = max(scores, key=scores.get)

    exp_item = current["expected_item"]

    # check if prediction is correct
    if exp_item == pred:
        total_acc += 1

final_acc = total_acc / len(data)
print(final_acc)

In [ ]:
# entire subtask B training data (20): 0.6

# only literal (7): 0.7142857142857143

# only idiomatic (13): 0.5384615384615384